In [ ]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
# %%writefile oil_price_prediction_app.py
import pandas as pd
import numpy as np
import streamlit as st
from math import sqrt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# Load the data
data = pd.read_csv("Crude oil.csv")  # Replace with the actual path
data.columns = ["Date", "Price", "Volume", "Open", "High", "Low"]

# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Drop unnecessary columns
columns_to_drop = ['Volume', 'Open', 'Low', 'High']
df = data.drop(columns=columns_to_drop)

# Model Fitting
st.title('Oil Price Prediction with ARIMA')

# Sidebar with date range selector
min_date = st.sidebar.date_input("Select start date", min(df['Date']))
max_date = st.sidebar.date_input("Select end date", max(df['Date']), min_date + pd.Timedelta(days=1))

# Convert min_date and max_date to datetime64[ns]
min_date = np.datetime64(min_date)
max_date = np.datetime64(max_date)

# Split the data based on selected date range
mask = (df['Date'] >= min_date) & (df['Date'] <= max_date)
selected_data = df.loc[mask]

# Display the DataFrame
st.subheader("Selected Data")
st.write(selected_data)

# ARIMA Model
train_size = int(len(selected_data) * 0.8)
train, test = selected_data[0:train_size], selected_data[train_size:]
order = (1, 1, 0)
seasonal_order = (1, 1, 1, 4)
model = SARIMAX(train['Price'], order=order, seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)

# Predictions
predictions = model_fit.predict(start=test.index[0], end=test.index[-1], dynamic=False)
rmse_Arima = sqrt(mean_squared_error(test['Price'], predictions))

# Plot actual vs predicted prices
st.subheader("Actual vs Predicted Prices")
st.line_chart(pd.DataFrame({'Actual': test['Price'], 'Predicted': predictions}))

# Display RMSE
st.subheader(f'Root Mean Squared Error (RMSE): {rmse_Arima}')

# Forecast
forecast_steps = st.slider("Number of Forecast Steps", 1, 30, 7)
forecast = model_fit.forecast(steps=forecast_steps)
st.subheader(f"Forecast for the next {forecast_steps} steps")
st.write(forecast)

# Weekly, Monthly, and Yearly Graphs
weekly_data = selected_data.resample('W-Mon', on='Date').mean()
monthly_data = selected_data.resample('M', on='Date').mean()
yearly_data = selected_data.resample('Y', on='Date').mean()

# Plot Weekly Graph
st.subheader("Weekly Prices")
st.line_chart(weekly_data['Price'])

# Plot Monthly Graph
st.subheader("Monthly Prices")
st.line_chart(monthly_data['Price'])

# Plot Yearly Graph
st.subheader("Yearly Prices")
st.line_chart(yearly_data['Price'])


2024-01-05 16:04:49.706 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [ ]:
#!npm install localtunnel
!wget -q -O - icanhazip.com

In [ ]:
!streamlit run oil_price_prediction_app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.292s
your url is: https://public-plants-win.loca.lt
